# Climatological daily-mean blue-sky surface albedo values
This notebook calculates ten-year climatologies based on hourly calculations of blue-sky surface albedo values for a specified land cover type for one 500 m grid cell. It follows the methodology outlined in Loog et al. (2026) to be able to derive values for multiple land cover types in the same location, allowing for the calculation of surface albedo changes which can then be included in climate-related impact assessments.

## Import necessary packages

In [1]:
import ee
import geemap
import geemap.colormaps as cm
import math
import pandas as pd

## Google Earth Engine authentication and initialization
Running this notebook requires a Google Earth Engine account with a registered project. If you are not already a Google Earth Engine user, you can register __[here](https://earthengine.google.com/)__.

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()

## Define location of interest
Enter latitude and longitude values

In [3]:
lat = 46.0156 # enter value
lon = -75.4211 # enter value

In [6]:
Map = geemap.Map(center=(lat, lon), 
                 zoom=8,
                 add_google_map=False
                )
Map.add_basemap('Esri.WorldStreetMap')
Map.add_marker([lat,lon])
Map

Map(center=[46.0156, -75.4211], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Load datasets
This defines and loads the datasets from Google Earth Engine required for the calculation

In [7]:
bbox = ee.Geometry.BBox(lon-0.5, lat-0.5, lon+0.5, lat+0.5)
albedo_BRDF = ee.ImageCollection("MODIS/061/MCD43A1").filterBounds(bbox)
albedo_scale = albedo_BRDF.first().projection().nominalScale()
lc_data = ee.ImageCollection('MODIS/061/MCD12Q1').filterBounds(bbox)
lc_quality = lc_data.select('QC').filterBounds(bbox)
lc_scale = lc_data.first().projection().nominalScale()
solar_in_all = ee.ImageCollection('ECMWF/ERA5/HOURLY').select('mean_surface_downward_short_wave_radiation_flux').filterBounds(bbox)
solar_in_direct = ee.ImageCollection('ECMWF/ERA5/HOURLY').select('mean_surface_direct_short_wave_radiation_flux').filterBounds(bbox)
solar_scale = solar_in_all.first().projection().nominalScale()

## Define functions

In [10]:
# Land cover change mask
# This function takes an image from the MODIS MCD12Q1 land cover dataset as an input and creates a mask that filters out 
# any data points where the land cover type is not the same in either the previous or following year

def change_mask(image):
    # Identify the start and end dates of the input image
    start_date = image.date()
    end_date = start_date.advance(1, 'year')

    # Identify start and end dates of the image from one year previous
    start_date_before = start_date.advance(-1, 'year')
    end_date_before = start_date
    
    # Identify start and end dates of the image from one year after
    start_date_after = end_date
    end_date_after = start_date_after.advance(1, 'year')
    
    # Select images from the year befreo and the year after based on the dates determined
    image_before = lc_data.select('LC_Type1').filterDate(start_date_before, end_date_before).first()
    image_after = lc_data.select('LC_Type1').filterDate(start_date_after, end_date_after).first()

    # Compare current image to the images from other years
    lc_change_mask_before = image.eq(image_before) # returns 1 where values are euqal in both years and 0 where unequal
    lc_change_mask_after = image.eq(image_after)
    
    # Combine before and after into one mask
    lc_change_mask = lc_change_mask_before.where(lc_change_mask_after.eq(0), 0) 
    return lc_change_mask

# Land cover spatial homogeneity mask
# This function takes an image from the MODIS MCD12Q1 land cover dataset as an input and creates a mask that filters out 
# any data points where the land cover type is not the same in the surrounding cells

def homog_mask(image):
     # Define the base land cover types
     lc_orig = image
     # Calculates the minimum of all the land cover types (expressed as integers) in the surrounding cells
     lc_min = image.reduceNeighborhood(reducer=ee.Reducer.min(),
                                       kernel=ee.Kernel.square(radius=1,
                                                               units='pixels'),
                                       inputWeight='mask')
     # Calculates the maximum of all the land cover types (expressed as integers) in the surrounding cells
     lc_max = image.reduceNeighborhood(reducer=ee.Reducer.max(),
                                       kernel=ee.Kernel.square(radius=1,
                                                               units='pixels'),
                                       inputWeight='mask')
     # If the min, max, and base land cover types are all the same value, this indicates that all surrounding cells are the same.
     # Where either the min or max value differs from the base land cover type, these values are masked out
     lc_homog_mask = lc_orig.where(lc_orig.eq(lc_min), 1)
     lc_homog_mask = lc_homog_mask.where(lc_orig.neq(lc_min), 0)
     lc_homog_mask = lc_homog_mask.where(lc_orig.neq(lc_max), 0)
     return lc_homog_mask

# Land cover data quality mask
# This function creates a mask based on the MODIS MCD12Q1 data quality flag

def qual_mask(image):
    lc_qual_x = image
    lc_qual_mask_x = ee.Image.constant(1).where(lc_qual_x.eq(1), 0).where(lc_qual_x.eq(2), 0).where(lc_qual_x.eq(3), 0).where(lc_qual_x.eq(4), 0).where(lc_qual_x.eq(5), 0).where(lc_qual_x.eq(7), 0)
    return lc_qual_mask_x

## Determine possible land cover types available at location
Note that the surface albedo for every land cover type cannot be determined at every location globally. This function will tell you the land cover types that it is possible to determine a surface albedo value for at the location specified

In [14]:
def available_lc_types(yr):
    start_date = ee.Date.fromYMD(yr, 1, 1)
    end_date = start_date.advance(1, 'year')
    # Calculate data quality mask by mapping masking function over the MODIS MCD12Q1 quality dataset corresponding to the same timeframe
    lc_qual_mask_x = lc_quality.filterDate(start_date, end_date).map(qual_mask).first()
    # Calculate land cover change mask by mapping masking function over the MODIS MCD12Q1 land cover dataset corresponding to the same timeframe
    lc_change_mask_x = lc_data.select('LC_Type1').filterDate(start_date, end_date).map(change_mask).first()
    # Calculate land cover spatial homogeneity mask by mapping masking function over the MODIS MCD12Q1 land cover dataset corresponding to the same timeframe
    lc_homog_mask_x = lc_data.select('LC_Type1').filterDate(start_date, end_date).map(homog_mask).first()

    lc_list_all = lc_data.filterDate('2011-01-01').first().select('LC_Type1').updateMask(lc_qual_mask_x).updateMask(lc_change_mask_x).reduceNeighborhood(reducer=ee.Reducer.fixedHistogram(min=1,max=18,steps=17),kernel=ee.Kernel.circle(radius=25*1000,units='meters'),skipMasked=False).reduceRegion(reducer=ee.Reducer.first(),geometry = ee.Geometry.Point([lon, lat]),scale = lc_scale)
    lc_types = lc_list_all.getInfo()
    histo_dict = dict(lc_types['LC_Type1_histogram'])
    histo_df = pd.DataFrame.from_dict(histo_dict,orient='index').rename(columns={0:str(yr)})
    return histo_df

histo_df = available_lc_types(2011)
for yr in range(2012,2021):
    histo_df = histo_df.join(available_lc_types(yr))

histo_df['sum'] = histo_df.sum(axis=1)
histo_df = histo_df[histo_df['sum']>0]

lc_avail_list = list(histo_df.index.values)

lc_type_dict = {1:'Evergreen Needleleaf Forests',2:'Evergreen Broadleaf Forests',3:'Deciduous Needleleaf Forests',4:'Deciduous Broadleaf Forests',5:'Mixed Forest',6:'Closed Shrublands',7:'Open Shrublands',8:'Woody Savannas',9:'Savannas',10:'Grasslands',11:'Permanent Wetlands',12:'Croplands',13:'Urban and Built-up Lands',14:'Cropland/Natural Vegetation Mosaics',16:'Barren/Non-Vegetated Lands'}

print('Available land cover types at this location: ')
for lc in lc_avail_list:
    print(str(lc) + ": " + lc_type_dict[lc])

Available land cover types at this location: 
1: Evergreen Needleleaf Forests
4: Deciduous Broadleaf Forests
5: Mixed Forest
8: Woody Savannas
9: Savannas
10: Grasslands
11: Permanent Wetlands
12: Croplands
13: Urban and Built-up Lands


## Specify land cover type of interest
Enter one of the values listed above to determine the surface albedo value.

In [15]:
land_cover_type = 4 #enter numberical land cover type here

## Calculation of daily-mean blue-sky surface albedo values, averaged monthly for ther period 2011-2020
Note that these culculations may take several minutes to complete.

In [16]:
# Function to calculate daily-mean blue-sky albedo calculation
# This is a series of nested functions that use global parameters. Although this is generally
# considered poor coding practice this was necessary to make use of the GEE mapping function which
# allows us to map one function across all the images of an image collection or all elements of a list. 
# One constraint of the mapping functionality is that only functions with a single input can be mapped.
# To work around this, we have created a nested function where upper levels provide necessary input 
# parameters to further nested functions

# Define hour angles
# Black-sky albedo is calculated every hour but on the half hour to correspond with the diffuse sylight data
# taken from the ERA5 dataset. A function to calculate hourly surface albedo values iterates over this list
hours = ee.List.sequence(start=0.5, end=23.5, step=1)

# Function to calculate the daily-mean blue-sy albedo values for a given land cover type for one month of the year
# yr is the year, d is sampling distance (in km), lc_in is the land cover type, m is the month of the year
# tile_scale is used to specify the tile scaling for GEE and sfx is a suffix that can be added to the output file
def albedo_bls_monthly(d, lc_in, m):
    def albedo_bls_yearly(yr):
        # Identify dates at beginning and end of year to select corresponding land cover data from the MODIS MCD12Q1
        # land cover dataset which is available on a yearly basis
        start_date = ee.Date.fromYMD(yr, 1, 1)
        end_date = start_date.advance(1, 'year')
        
        #Band selection for the MODIS MCD12Q1. Depending on the land cover type selected in the input parameters different 
        #bands are required to retrieve the data
        if lc_in == 12.7:
            band = 'LC_Type5'
            lc = 7
        elif lc_in == 12.8:
            band = 'LC_Type5'
            lc = 8
        elif lc_in == 14.25:
            band = 'LC_Prop2'
            lc = 25
        elif lc_in == 14.35:
            band = 'LC_Prop2'
            lc = 35
        else:
            band = 'LC_Type1'
            lc = lc_in
    
        def albedo_bls_daily(image):
            # This function iterates over the MODIS MCD43A1 dataset (images are daily) and calculates the daily-mean blue-sky albedo
            # Extract starting date of the image to select images from the same date across datasets
            start_time = ee.Date(image.get('system:time_start'))
            end_time = start_time.advance(1, 'day')
    
            # BRDF model parameters from the MODIS MCD43A1 dataset
            BRDF_iso = image.select('BRDF_Albedo_Parameters_shortwave_iso').multiply(0.001)
            BRDF_vol = image.select('BRDF_Albedo_Parameters_shortwave_vol').multiply(0.001)
            BRDF_geo = image.select('BRDF_Albedo_Parameters_shortwave_geo').multiply(0.001)
    
            # Calculation of white-sky albedo from the BRDF model parameters
            albedo_ws_x = BRDF_iso.add(BRDF_vol.multiply(0.189184)).add(BRDF_geo.multiply(-1.377622))
    
            # Calculate hourly diffuse skylight ratios
            solar_in_x = solar_in_all.filterDate(start_time,end_time).toBands()
            solar_in_dir_x = solar_in_direct.filterDate(start_time,end_time).toBands()
            
            dsr_x = (solar_in_x.subtract(solar_in_dir_x)).divide(solar_in_x)
            dsr_inv_x = solar_in_dir_x.divide(solar_in_x)
            
            dsr_mask = dsr_x.where(dsr_x.lt(0),0).where(dsr_x.gte(0),1).where(dsr_x.gt(1),0)
            dsr_inv_mask = dsr_inv_x.where(dsr_inv_x.lt(0),0).where(dsr_inv_x.gte(0),1).where(dsr_inv_x.gt(1),0)
            
            dsr_x = dsr_x.updateMask(dsr_mask)
            dsr_inv_x = dsr_x.updateMask(dsr_inv_mask)
            
            # Define additional parameters needed to calculate the blue sky albedo
            # Longitude (in degrees)
            lon = image.pixelCoordinates(projection='EPSG:4326').select('x')
            # Latitude (in radians)
            lat = image.pixelCoordinates(projection='EPSG:4326').select('y').multiply(math.pi).divide(180)
            #Day of year
            DOY = start_time.getRelative('day', 'year')
            # Solar declination angle (based on the day of year)
            SDA = (DOY.add(284)).multiply(360).divide(365.24).multiply(math.pi).divide(180).sin().multiply(23.44).multiply(math.pi).divide(180)# in radians
            
    
            # Function to calculate the cosine of the solar zenith angle (SZA) from UTC time (input is a constant)
            def cos_SZA_calc(hour):
                # Convert input UTC time to image
                UTC_time_x = ee.Image.constant(hour)
                # Calculate solar time for each location based on the longitute and UTC time
                solar_time_x = UTC_time_x.add(lon.divide(15))
                solar_time_x = solar_time_x.where(solar_time_x.lt(0), solar_time_x.add(24))
                # Calculate hour angles from solar time
                hour_angles_x = (solar_time_x.subtract(ee.Number(12))).multiply(15).divide(180).multiply(math.pi)
    
                # Calculate cos(SZA) from hour angles
                cos_SZA_x = lat.sin().multiply(SDA.sin()).add(lat.cos().multiply(SDA.cos()).multiply(hour_angles_x.cos()))
                # Mask where cos(SZA) is less than 0
                mask = cos_SZA_x.mask().where(cos_SZA_x.lt(0), 0)
                cos_SZA_x = cos_SZA_x.updateMask(mask)
                return cos_SZA_x
            
            # Map function across the hours list to get the global solar zenith angles for all hours of the day
            cos_SZA_hourly = ee.ImageCollection.fromImages(hours.map(cos_SZA_calc))
    
            # Function to calculate the instantaneous black-sky albedo based on the cosine of the SZA
            def BKS_calc(cos_sza_x):
                # Define solar zenith angle to input into BRDF polynomial function
                SZA = cos_sza_x.acos()
                # Calculate the three terms of the BRDF polynomial (iso, vol, and geo)
                BRDF_term_1 = BRDF_iso
                BRDF_term_2 = BRDF_vol.multiply(ee.Image.constant(-0.007574).add(SZA.multiply(SZA).multiply(-0.070987)).add(SZA.multiply(SZA).multiply(SZA).multiply(0.307588)))
                BRDF_term_3 = BRDF_geo.multiply(ee.Image.constant(-1.284909).add(SZA.multiply(SZA).multiply(-0.166314)).add(SZA.multiply(SZA).multiply(SZA).multiply(0.041840)))
    
                # Sum the terms of the BRDF polynomial to get the instantaneous black-sky albedo
                bks_alb_x = BRDF_term_1.add(BRDF_term_2).add(BRDF_term_3)
    
                # Mask out any anomolous data (greater than 1 or less than 0)
                albedo_bks_mask = bks_alb_x.where(bks_alb_x.lte(0), 0)  # invalid
                albedo_bks_mask = albedo_bks_mask.where(bks_alb_x.gt(0), 1)  # valid 
                albedo_bks_mask = albedo_bks_mask.where(bks_alb_x.gte(1), 0)  # invalid
                bks_alb_x = bks_alb_x.updateMask(albedo_bks_mask)
                return bks_alb_x
            
            # Map function across all solar zenith angles to get global hourly black sky albedo values and
            # convert to image with same band names as the image containing the diffuse skylight ratios
            bands_hourly = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 
                            'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19', 'b20', 'b21', 'b22', 'b23', 'b24']
            
            albedo_bks_x = cos_SZA_hourly.map(BKS_calc).toBands().rename(bands_hourly)
            
            # Mask out and anomolous data (greater than 1 or less than 0)
            albedo_ws_mask = albedo_ws_x.mask().where(albedo_ws_x.lte(0), 0)  # invalid
            albedo_ws_mask = albedo_ws_mask.where(albedo_ws_x.gte(1), 0)  # invalid
            albedo_ws_x = albedo_ws_x.updateMask(albedo_ws_mask)
            
            # Mask out high solar zenith angles
            SZA_noon = lat.sin().multiply(SDA.sin()).add(lat.cos().multiply(SDA.cos())).acos().multiply(180).divide(math.pi)
            SZA_mask = SZA_noon.mask().where(SZA_noon.gt(75),0)
            albedo_bks_x = albedo_bks_x.updateMask(SZA_mask)
    
            # Calculate blue-sky albedo values
            # Hourly blue-sky values calculate =s from the daily white-sky albedo and hourly black-sky values and hourly DSR values
            albedo_BLS_x = ((albedo_bks_x.multiply(dsr_inv_x)).add(albedo_ws_x.multiply(dsr_x)))
            
            # Determine weighting factors to calcualte daily-average blue sky values (weighted by the incoming solar radiation)
            # Total daily solar insolation
            tot_daily_solar = cos_SZA_hourly.toBands().updateMask(dsr_x.mask()).reduce(reducer=ee.Reducer.sum()).multiply(1361)
            # Hourly solar insolation divided by total daily solar to give fractional weighting for each hour of the day
            wtg_factor = cos_SZA_hourly.toBands().updateMask(dsr_x.mask()).rename(bands_hourly).multiply(1361).divide(tot_daily_solar)
            
            # Daily-mean blue-sky albedo values
            albedo_BLS_mean = albedo_BLS_x.multiply(wtg_factor).reduce(ee.Reducer.sum().unweighted())
    
            return albedo_BLS_mean
    
        # Calculate monthly average albedo values by mapping the daily-mean blue-sky albedo function across one month of data
        albedo_bls_m = albedo_BRDF.filterDate(start_date, end_date).filter(ee.Filter.calendarRange(m, m, 'month')).map(albedo_bls_daily).mean()
    
        # Isolate per land cover type by masking out all other land cover types
        lc_igbp_x = lc_data.select(band).filterDate(start_date, end_date).first()
        
        if lc == 'forest':
            lc_type_mask_x = lc_igbp_x.where(lc_igbp_x.neq(1), 0).where(lc_igbp_x.eq(1), 1).where(lc_igbp_x.eq(2), 1).where(lc_igbp_x.eq(3), 1).where(lc_igbp_x.eq(4), 1).where(lc_igbp_x.eq(5), 1)
            albedo_bls_m = albedo_bls_m.updateMask(lc_type_mask_x)
            
        elif lc == 11.27 or lc == 11.50:
            lc_type_mask_x = lc_igbp_x.where(lc_igbp_x.neq(11), 0).where(lc_igbp_x.eq(11), 1)
            albedo_bls_m = albedo_bls_m.updateMask(lc_type_mask_x)
            if lc == 11.27:
                lc_prop_x = lc_data.select('LC_Prop3').filterDate(start_date, end_date).first()
                lc_prop_type_mask_x = lc_prop_x.where(lc_igbp_x.neq(27), 0).where(lc_prop_x.eq(27), 1)
                albedo_bls_m = albedo_bls_m.updateMask(lc_prop_type_mask_x)
                
            if lc == 11.50:
                lc_prop_x = lc_data.select('LC_Prop3').filterDate(start_date, end_date).first()
                lc_prop_type_mask_x = lc_prop_x.where(lc_igbp_x.neq(50), 0).where(lc_prop_x.eq(50), 1)
                albedo_bls_m = albedo_bls_m.updateMask(lc_prop_type_mask_x)
                
        else:
            lc_type_mask_x = lc_igbp_x.where(lc_igbp_x.neq(lc), 0).where(lc_igbp_x.eq(lc), 1)
            albedo_bls_m = albedo_bls_m.updateMask(lc_type_mask_x)
            
        # Calculate and land cover masks
        # Calculate data quality mask by mapping masking function over the MODIS MCD12Q1 quality dataset corresponding to the same timeframe
        lc_qual_mask_x = lc_quality.filterDate(start_date, end_date).map(qual_mask).first()
        # Calculate land cover change mask by mapping masking function over the MODIS MCD12Q1 land cover dataset corresponding to the same timeframe
        lc_change_mask_x = lc_data.select('LC_Type1').filterDate(start_date, end_date).map(change_mask).first()
        # Calculate land cover spatial homogeneity mask by mapping masking function over the MODIS MCD12Q1 land cover dataset corresponding to the same timeframe
        lc_homog_mask_x = lc_data.select('LC_Type1').filterDate(start_date, end_date).map(homog_mask).first()
        
        # Apply land cover masks
        albedo_bls_m = albedo_bls_m.updateMask(lc_qual_mask_x).updateMask(lc_change_mask_x)
    
        # Apply spatial averaging, extending values by the distance specified by d
        albedo_bls_m = albedo_bls_m.reduceNeighborhood(reducer=ee.Reducer.mean(),kernel=ee.Kernel.circle(radius=d*1000,units='meters'),skipMasked=False).reproject('EPSG:4326', scale=lc_scale)
        return albedo_bls_m
    years = ee.List.sequence(start = 2011,end = 2020)
    alb_all = ee.ImageCollection(years.map(albedo_bls_yearly)).mean()
    return alb_all

# Function to calculate the monthyl incoming solar radiation
def solar_rad_monthly(m):
    def solar_rad_yearly(yr):
        start_date = ee.Date.fromYMD(yr, m, 1)
        end_date = start_date.advance(1, 'month')
        solar_in_yearly = solar_in_all.filterBounds(ee.Geometry.Point([lon, lat])).filterDate(start_date, end_date).sum()
        return solar_in_yearly
    years_list = ee.List.sequence(2011,2020)
    yearly_solar = ee.ImageCollection(years_list.map(solar_rad_yearly)).mean()
    
    return yearly_solar

d = 25
month_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

print('10-year climatological surface albedo values for '+lc_type_dict[land_cover_type]+' at (' + str(lat) + ', ' + str(lon) + '):')
alb_list = []
solar_list = []
na_flag = 0
for m in range(1,13):
    alb = albedo_bls_monthly(d, land_cover_type, m).reduceRegion(reducer=ee.Reducer.mean(),geometry = ee.Geometry.Point([lon, lat]),scale = lc_scale).getInfo()
    alb_list.append(alb['sum_mean'])
    if alb['sum_mean'] == None:
        na_flag = 1
    
    solar_year_list = []
    for year in range(2011,2021):
        start_date = ee.Date.fromYMD(year, m, 1)
        end_date = start_date.advance(1, 'month')
        solar_m = solar_in_all.filterDate(start_date,end_date).sum().reduceRegion(reducer=ee.Reducer.mean(),geometry = ee.Geometry.Point([lon, lat]),scale = solar_scale, tileScale=16).getInfo()
        solar_year_list.append(solar_m['mean_surface_downward_short_wave_radiation_flux'])
        solar_m = sum(solar_year_list) / len(solar_year_list)
    solar_list.append(solar_m)
    print(month_list[m-1] + ': ' + str(alb['sum_mean']))

if na_flag == 1:
    print('Yearly average: insufficient data')
else:
    yearly_avg_df = pd.DataFrame(data={'alb_vals':alb_list,'solar_ins':solar_list})
    yearly_avg_df['alb_weighted'] = yearly_avg_df.alb_vals * yearly_avg_df.solar_ins / yearly_avg_df.solar_ins.sum()
    year_avg = yearly_avg_df['alb_weighted'].sum()
    print('Yearly average: ' + str(year_avg))

10-year climatological surface albedo values for Deciduous Broadleaf Forests at (46.0156, -75.4211):
Jan: 0.29177558667561326
Feb: 0.29567671508839516
Mar: 0.24700986286424262
Apr: 0.15974068637097705
May: 0.11152626536690413
Jun: 0.1381128877282243
Jul: 0.11714477219601442
Aug: 0.12941275046398437
Sep: 0.13098759042933764
Oct: 0.14130485674316579
Nov: 0.1346155075084596
Dec: 0.2619684455078958
Yearly average: 0.15923843966899354
